## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [ ]:
# If you didn't download it yet, please get the relevant file now!
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ~/data/

1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

In [51]:
import pandas as pd
import numpy as np

d = pd.read_csv("~/data/data_000637.txt")
N = np.random.randint(10000, d.shape[0])

d.head(N)


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
792760,1,1,104,3869207628,1728,5
792761,1,1,109,3869207628,1722,26
792762,1,1,110,3869207628,1726,27
792763,1,1,109,3869207628,1730,17


2\. Find out the number of BX in a ORBIT (the value 'x').

In [52]:
bx_counter = d.groupby("ORBIT_CNT")["BX_COUNTER"].max()
print(bx_counter)

print(bx_counter.max())

ORBIT_CNT
3869200167    3187
3869200168    3538
3869200169    2766
3869200170    3377
3869200171    3542
              ... 
3869211167    3553
3869211168    3556
3869211169    3498
3869211170    3527
3869211171     769
Name: BX_COUNTER, Length: 11001, dtype: int64
3563


3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

In [58]:
total_time = bx_counter.mean() * 25 * bx_counter.shape[0]
print('Estimated total time (ns):', int(total_time))
print('Total time:', int(bx_counter.max() * 25 * bx_counter.shape[0]))

Estimated total time (ns): 902131900
Total time: 979914075


4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

In [70]:
indexes = []
a = 0

for i in range(1, bx_counter.shape[0]):
    if d.loc[[i],['ORBIT_CNT']] != d.loc[[i - 1],['ORBIT_CNT']]:
        a += a
    
    indexes.append(a)
    
print(len(indexes), indexes)

d['TOTAL_TIME'] =  (d['BX_COUNTER'] * 25 + d['TDC_MEAS'] * 25/30)
d


ValueError: Can only compare identically-labeled (both index and columns) DataFrame objects

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

6\. Create a new DataFrame that contains only the rows with HEAD=1.

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139